[![Binder](../img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignments/assignment-signal-portfolio.ipynb)&emsp;
[![Script](../img/badge-script.svg)](/Teaching//assignments/assignment-signal-portfolio.fsx)&emsp;
[![Notebook](../img/badge-notebook.svg)](/Teaching//assignments/assignment-signal-portfolio.ipynb)

Group Name:

Student Name | Student Number
--- | ---
**1:Bernardo Manarte** |55810 &#32;
**2:Leonor Rodrigues Pereira** |48778 &#32;
**3:Marouan Kamoun** |53833 &#32;
**4:Rodrigo Simões** |53154 &#32;
**5:Vasco Calxito** |53960 &#32;


This is an assignment. You should work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.

For testing



In [3]:
#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
open Xunit
open FsUnit.Xunit
open FsUnitTyped


Installed Packages FsUnit.Xunit, 5.3.0 xunit, 2.4.2

For the assignment



In [14]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: Plotly.NET"

Installed Packages FSharp.Data, 6.2.0 FSharp.Stats, 0.4.11 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 plotly.net, 3.0.0

In [5]:
#r "nuget: Plotly.NET.Interactive, 3.*"


Installed Packages FSharp.Data, 6.2.0 FSharp.Stats, 0.4.11 Plotly.NET.Interactive, 3.0.2

Loading extensions from `/Users/rodrigosimoes/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

In [15]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open MathNet.Numerics.Statistics


## Load Data

First, make sure that you're referencing the correct files.

Here I'm assuming that you have a class folder with this notebook and a `data` folder inside of it. The folder hierarchy would look like below where you
have the below files and folders accessible:

```code
/class
    notebook.ipynb
    id_and_return_data.csv
    be_me.csv
    
```


In [16]:
open NovaSBE.Finance.Portfolio


### Data files

We assume the `id_and_return_data.csv` file and the signal csv file  are in the `data` folder. In this example the signal file is `be_me.csv`.



In [4]:
let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "be_me.csv"
let strategyName = "book-to-market"


If my paths are correct, then this code should read the first few lines of the files.
If it doesn't show the first few lines, fix the above file paths.



In [19]:
IO.File.ReadLines(IdAndReturnsFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


"id(string),eom(date),source(string),sizeGrp(string),obsMain(bool),exchMain(bool),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option)"
"crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875"
"crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954"
"crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511210,36,0.285714,0.28137

In [21]:
IO.File.ReadLines(MySignalFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


"id(string),eom(date),signal(float option)"
"comp_001034_01,2008-12-31T00:00:00.0000000,0.5559700602"
"comp_001043_01,2000-01-31T00:00:00.0000000,"
"comp_001076_02,2010-12-31T00:00:00.0000000,0.6593168365"
"comp_001081_01,2007-10-31T00:00:00.0000000,2.7828229254"


Ok, now assuming those paths were correct the below code will work.
I will put all this prep code in one block so that it is easy to run.



In [22]:
let idAndReturnsCsv = 
    CsvProvider<IdAndReturnsFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList
let mySignalCsv = 
    CsvProvider<MySignalFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList


In [23]:
mySignalCsv

index value 0 (comp_001034_01, 31/12/2008 00:00:00, Some(0.5559700602)) Item1 comp_001034_01 Item2 2008-12-31 00:00:00Z Item3 Some(0.5559700602) Value 0.5559700602 1 (comp_001043_01, 31/01/2000 00:00:00, ) Item1 comp_001043_01 Item2 2000-01-31 00:00:00Z Item3 <null> 2 (comp_001076_02, 31/12/2010 00:00:00, Some(0.6593168365)) Item1 comp_001076_02 Item2 2010-12-31 00:00:00Z Item3 Some(0.6593168365) Value 0.6593168365 3 (comp_001081_01, 31/10/2007 00:00:00, Some(2.7828229254)) Item1 comp_001081_01 Item2 2007-10-31 00:00:00Z Item3 Some(2.7828229254) Value 2.7828229254 4 (comp_001095_01, 30/06/2002 00:00:00, Some(0.6236514886)) Item1 comp_001095_01 Item2 2002-06-30 00:00:00Z Item3 Some(0.6236514886) Value 0.6236514886 5 (comp_001111_02, 31/07/2008 00:00:00, Some(0.2180964901)) Item1 comp_001111_02 Item2 2008-07-31 00:00:00Z Item3 Some(0.2180964901) Value 0.2180964901 6 (comp_001117_02, 31/05/2003 00:00:00, Some(1.0342302181)) Item1 comp_001117_02 Item2 2003-05-31 00:00:00Z Item3 Some(1.0342302181) Value 1.0342302181 7 (comp_001117_02, 30/06/2003 00:00:00, Some(0.8751178769)) Item1 comp_001117_02 Item2 2003-06-30 00:00:00Z Item3 Some(0.8751178769) Value 0.8751178769 8 (comp_001117_02, 31/07/2003 00:00:00, Some(0.8417513135)) Item1 comp_001117_02 Item2 2003-07-31 00:00:00Z Item3 Some(0.8417513135) Value 0.8417513135 9 (comp_001117_02, 31/08/2003 00:00:00, Some(0.809376263)) Item1 comp_001117_02 Item2 2003-08-31 00:00:00Z Item3 Some(0.809376263) Value 0.809376263 10 (comp_001117_02, 30/09/2003 00:00:00, Some(0.3288091068)) Item1 comp_001117_02 Item2 2003-09-30 00:00:00Z Item3 Some(0.3288091068) Value 0.3288091068 11 (comp_001117_02, 31/10/2003 00:00:00, Some(0.3399090198)) Item1 comp_001117_02 Item2 2003-10-31 00:00:00Z Item3 Some(0.3399090198) Value 0.3399090198 12 (comp_001117_02, 30/11/2003 00:00:00, Some(0.3381757213)) Item1 comp_001117_02 Item2 2003-11-30 00:00:00Z Item3 Some(0.3381757213) Value 0.3381757213 13 (comp_001117_02, 31/12/2003 00:00:00, Some(0.3294486059)) Item1 comp_001117_02 Item2 2003-12-31 00:00:00Z Item3 Some(0.3294486059) Value 0.3294486059 14 (comp_001117_02, 31/01/2004 00:00:00, Some(0.2036921396)) Item1 comp_001117_02 Item2 2004-01-31 00:00:00Z Item3 Some(0.2036921396) Value 0.2036921396 15 (comp_001117_02, 29/02/2004 00:00:00, Some(0.1481397379)) Item1 comp_001117_02 Item2 2004-02-29 00:00:00Z Item3 Some(0.1481397379) Value 0.1481397379 16 (comp_001117_02, 31/03/2004 00:00:00, Some(0.2060292878)) Item1 comp_001117_02 Item2 2004-03-31 00:00:00Z Item3 Some(0.2060292878) Value 0.2060292878 17 (comp_001117_02, 30/04/2004 00:00:00, Some(0.1795151741)) Item1 comp_001117_02 Item2 2004-04-30 00:00:00Z Item3 Some(0.1795151741) Value 0.1795151741 18 (comp_001117_02, 31/05/2004 00:00:00, Some(0.1944747719)) Item1 comp_001117_02 Item2 2004-05-31 00:00:00Z Item3 Some(0.1944747719) Value 0.1944747719 19 (comp_001117_02, 30/06/2004 00:00:00, Some(0.1897314848)) Item1 comp_001117_02 Item2 2004-06-30 00:00:00Z Item3 Some(0.1897314848) Value 0.1897314848 ... (more)

A list of `Signal` records. The signal type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L178-L181).



In [10]:
let mySignals =
    mySignalCsv
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal }
            Some signalRecord)

// look at a few signals
mySignals[..3]


index value 0 { SecurityId = Other "comp_001034_01"\n FormationDate = 01/12/2008 00:00:00\n Signal = 0.5559700602 } SecurityId Other "comp_001034_01" Item comp_001034_01 FormationDate 2008-12-01 00:00:00Z Signal 0.5559700602 1 { SecurityId = Other "comp_001076_02"\n FormationDate = 01/12/2010 00:00:00\n Signal = 0.6593168365 } SecurityId Other "comp_001076_02" Item comp_001076_02 FormationDate 2010-12-01 00:00:00Z Signal 0.6593168365 2 { SecurityId = Other "comp_001081_01"\n FormationDate = 01/10/2007 00:00:00\n Signal = 2.782822925 } SecurityId Other "comp_001081_01" Item comp_001081_01 FormationDate 2007-10-01 00:00:00Z Signal 2.7828229254 3 { SecurityId = Other "comp_001095_01"\n FormationDate = 01/06/2002 00:00:00\n Signal = 0.6236514886 } SecurityId Other "comp_001095_01" Item comp_001095_01 FormationDate 2002-06-01 00:00:00Z Signal 0.6236514886

A list of Security return records. The `SecurityReturn` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L173-L176)



In [11]:
let myReturns =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

// look at a few returns
myReturns[..3]


index value 0 { SecurityId = Other "crsp_86432"\n Date = 01/01/2000 00:00:00\n Return = -0.003906 } SecurityId Other "crsp_86432" Item crsp_86432 Date 2000-01-01 00:00:00Z Return -0.003906 1 { SecurityId = Other "crsp_85640"\n Date = 01/01/2000 00:00:00\n Return = -0.157143 } SecurityId Other "crsp_85640" Item crsp_85640 Date 2000-01-01 00:00:00Z Return -0.157143 2 { SecurityId = Other "crsp_86430"\n Date = 01/01/2000 00:00:00\n Return = 0.285714 } SecurityId Other "crsp_86430" Item crsp_86430 Date 2000-01-01 00:00:00Z Return 0.285714 3 { SecurityId = Other "crsp_85756"\n Date = 01/01/2000 00:00:00\n Return = 0.190685 } SecurityId Other "crsp_85756" Item crsp_85756 Date 2000-01-01 00:00:00Z Return 0.190685

A list of security market caps. We'll need this for value-weight portfolios. The `WeightVariable` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L183-L186).



In [12]:
let myMktCaps =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

// look at a few market caps
myMktCaps[..3]


index value 0 { SecurityId = Other "crsp_86432"\n FormationDate = 01/01/2000 00:00:00\n Value = 83.161875 } SecurityId Other "crsp_86432" Item crsp_86432 FormationDate 2000-01-01 00:00:00Z Value 83.161875 1 { SecurityId = Other "crsp_85640"\n FormationDate = 01/01/2000 00:00:00\n Value = 377.954 } SecurityId Other "crsp_85640" Item crsp_85640 FormationDate 2000-01-01 00:00:00Z Value 377.954 2 { SecurityId = Other "crsp_86430"\n FormationDate = 01/01/2000 00:00:00\n Value = 42.38325 } SecurityId Other "crsp_86430" Item crsp_86430 FormationDate 2000-01-01 00:00:00Z Value 42.38325 3 { SecurityId = Other "crsp_85756"\n FormationDate = 01/01/2000 00:00:00\n Value = 16.2155 } SecurityId Other "crsp_85756" Item crsp_85756 FormationDate 2000-01-01 00:00:00Z Value 16.2155

## Forming our strategy

We're now going to use the `Backtest` code to generate portfolios. It is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L199).
The `Backtest` class automates some of the code we did earlier to make portfolio construction simpler.



In [13]:
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)


### Equal Weighted Portfolios



In [14]:
let ew = backtest.strategyEqualWeighted()


Some portfolios with their positions.



In [15]:
ew.Portfolios[..3]


index value 0 { Name = "book-to-market"\n Index = 1\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_92800"\n Weight = 0.0006451612903 }; { SecurityId = Other "crsp_83386"\n Weight = 0.0006451612903 };\n { SecurityId = Other "crsp_8... Name book-to-market Index 1 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_92800"\n Weight = 0.0006451612903 } SecurityId Other "crsp_92800" Item crsp_92800 Weight 0.0006451612903225806 1 { SecurityId = Other "crsp_83386"\n Weight = 0.0006451612903 } SecurityId Other "crsp_83386" Item crsp_83386 Weight 0.0006451612903225806 2 { SecurityId = Other "crsp_82567"\n Weight = 0.0006451612903 } SecurityId Other "crsp_82567" Item crsp_82567 Weight 0.0006451612903225806 3 { SecurityId = Other "crsp_90488"\n Weight = 0.0006451612903 } SecurityId Other "crsp_90488" Item crsp_90488 Weight 0.0006451612903225806 4 { SecurityId = Other "comp_180759_01"\n Weight = 0.0006451612903 } SecurityId Other "comp_180759_01" Item comp_180759_01 Weight 0.0006451612903225806 5 { SecurityId = Other "crsp_79698"\n Weight = 0.0006451612903 } SecurityId Other "crsp_79698" Item crsp_79698 Weight 0.0006451612903225806 6 { SecurityId = Other "crsp_88159"\n Weight = 0.0006451612903 } SecurityId Other "crsp_88159" Item crsp_88159 Weight 0.0006451612903225806 7 { SecurityId = Other "crsp_90325"\n Weight = 0.0006451612903 } SecurityId Other "crsp_90325" Item crsp_90325 Weight 0.0006451612903225806 8 { SecurityId = Other "crsp_85706"\n Weight = 0.0006451612903 } SecurityId Other "crsp_85706" Item crsp_85706 Weight 0.0006451612903225806 9 { SecurityId = Other "crsp_87816"\n Weight = 0.0006451612903 } SecurityId Other "crsp_87816" Item crsp_87816 Weight 0.0006451612903225806 10 { SecurityId = Other "crsp_91434"\n Weight = 0.0006451612903 } SecurityId Other "crsp_91434" Item crsp_91434 Weight 0.0006451612903225806 11 { SecurityId = Other "crsp_79678"\n Weight = 0.0006451612903 } SecurityId Other "crsp_79678" Item crsp_79678 Weight 0.0006451612903225806 12 { SecurityId = Other "crsp_89952"\n Weight = 0.0006451612903 } SecurityId Other "crsp_89952" Item crsp_89952 Weight 0.0006451612903225806 13 { SecurityId = Other "crsp_84020"\n Weight = 0.0006451612903 } SecurityId Other "crsp_84020" Item crsp_84020 Weight 0.0006451612903225806 14 { SecurityId = Other "crsp_81128"\n Weight = 0.0006451612903 } SecurityId Other "crsp_81128" Item crsp_81128 Weight 0.0006451612903225806 15 { SecurityId = Other "crsp_76605"\n Weight = 0.0006451612903 } SecurityId Other "crsp_76605" Item crsp_76605 Weight 0.0006451612903225806 16 { SecurityId = Other "crsp_91461"\n Weight = 0.0006451612903 } SecurityId Other "crsp_91461" Item crsp_91461 Weight 0.0006451612903225806 17 { SecurityId = Other "crsp_90169"\n Weight = 0.0006451612903 } SecurityId Other "crsp_90169" Item crsp_90169 Weight 0.0006451612903225806 18 { SecurityId = Other "crsp_85216"\n Weight = 0.0006451612903 } SecurityId Other "crsp_85216" Item crsp_85216 Weight 0.0006451612903225806 19 { SecurityId = Other "crsp_90061"\n Weight = 0.0006451612903 } SecurityId Other "crsp_90061" Item crsp_90061 Weight 0.0006451612903225806 ... (more) 1 { Name = "book-to-market"\n Index = 2\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_90454"\n Weight = 0.0006455777921 }; { SecurityId = Other "crsp_23712"\n Weight = 0.0006455777921 };\n { SecurityId = Other "crsp_1... Name book-to-market Index 2 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_90454"\n Weight = 0.0006455777921 } SecurityId Other "crsp_90454" Item crsp_90454 Weight 0.0006455777921239509 1 { SecurityId = Other "crsp_23712"\n Weight = 0.0006455777921 } SecurityId Other "crsp_23712" Item crsp_23712 Weight 0.0006455777921239509 2 { SecurityId = Other "crsp_18411"\n Weight = 0.0006455777921 } SecurityId Other "crsp_18411" Item crsp_18411 Weight 0.0006455777921239509 3 { SecurityId = Other "crsp_42439"\n Weight = 

Some portfolio returns.



In [16]:
ew.Returns[..3]


index value 0 { Name = "book-to-market"\n Index = 1\n Month = 01/01/2009 00:00:00\n Return = -0.04363115613 } Name book-to-market Index 1 Month 2009-01-01 00:00:00Z Return -0.04363115612903222 1 { Name = "book-to-market"\n Index = 2\n Month = 01/01/2009 00:00:00\n Return = -0.05140669141 } Name book-to-market Index 2 Month 2009-01-01 00:00:00Z Return -0.05140669141381543 2 { Name = "book-to-market"\n Index = 3\n Month = 01/01/2009 00:00:00\n Return = 0.00328639122 } Name book-to-market Index 3 Month 2009-01-01 00:00:00Z Return 0.0032863912201420284 3 { Name = "book-to-market"\n Index = 1\n Month = 01/01/2011 00:00:00\n Return = 0.004434853989 } Name book-to-market Index 1 Month 2011-01-01 00:00:00Z Return 0.0044348539886039885

### Value Weighted Portfolios



In [17]:
let vw = backtest.strategyValueWeighted(myMktCaps)

vw.Portfolios[..3]


index value 0 { Name = "book-to-market"\n Index = 1\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_92800"\n Weight = 7.82337602e-06 }; { SecurityId = Other "crsp_83386"\n Weight = 2.021589824e-05 };\n { SecurityId = Other "crsp_825... Name book-to-market Index 1 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_92800"\n Weight = 7.82337602e-06 } SecurityId Other "crsp_92800" Item crsp_92800 Weight 7.823376019658486E-06 1 { SecurityId = Other "crsp_83386"\n Weight = 2.021589824e-05 } SecurityId Other "crsp_83386" Item crsp_83386 Weight 2.021589824150812E-05 2 { SecurityId = Other "crsp_82567"\n Weight = 4.651873303e-05 } SecurityId Other "crsp_82567" Item crsp_82567 Weight 4.651873303407826E-05 3 { SecurityId = Other "crsp_90488"\n Weight = 2.805908036e-06 } SecurityId Other "crsp_90488" Item crsp_90488 Weight 2.805908036166525E-06 4 { SecurityId = Other "comp_180759_01"\n Weight = 1.370770344e-05 } SecurityId Other "comp_180759_01" Item comp_180759_01 Weight 1.370770344343533E-05 5 { SecurityId = Other "crsp_79698"\n Weight = 0.0002423434367 } SecurityId Other "crsp_79698" Item crsp_79698 Weight 0.00024234343666512957 6 { SecurityId = Other "crsp_88159"\n Weight = 7.712778478e-05 } SecurityId Other "crsp_88159" Item crsp_88159 Weight 7.71277847807716E-05 7 { SecurityId = Other "crsp_90325"\n Weight = 2.678371234e-05 } SecurityId Other "crsp_90325" Item crsp_90325 Weight 2.6783712335954458E-05 8 { SecurityId = Other "crsp_85706"\n Weight = 1.482623975e-05 } SecurityId Other "crsp_85706" Item crsp_85706 Weight 1.4826239745163141E-05 9 { SecurityId = Other "crsp_87816"\n Weight = 5.104974214e-05 } SecurityId Other "crsp_87816" Item crsp_87816 Weight 5.1049742144116546E-05 10 { SecurityId = Other "crsp_91434"\n Weight = 9.046313474e-05 } SecurityId Other "crsp_91434" Item crsp_91434 Weight 9.046313474129236E-05 11 { SecurityId = Other "crsp_79678"\n Weight = 0.00162203822 } SecurityId Other "crsp_79678" Item crsp_79678 Weight 0.001622038219962141 12 { SecurityId = Other "crsp_89952"\n Weight = 0.0002748332582 } SecurityId Other "crsp_89952" Item crsp_89952 Weight 0.0002748332581892655 13 { SecurityId = Other "crsp_84020"\n Weight = 0.000398395727 } SecurityId Other "crsp_84020" Item crsp_84020 Weight 0.000398395726968492 14 { SecurityId = Other "crsp_81128"\n Weight = 0.0005310074374 } SecurityId Other "crsp_81128" Item crsp_81128 Weight 0.0005310074373897382 15 { SecurityId = Other "crsp_76605"\n Weight = 0.001158061064 } SecurityId Other "crsp_76605" Item crsp_76605 Weight 0.0011580610635797482 16 { SecurityId = Other "crsp_91461"\n Weight = 0.001482692028 } SecurityId Other "crsp_91461" Item crsp_91461 Weight 0.0014826920280422897 17 { SecurityId = Other "crsp_90169"\n Weight = 2.806264885e-06 } SecurityId Other "crsp_90169" Item crsp_90169 Weight 2.806264885396256E-06 18 { SecurityId = Other "crsp_85216"\n Weight = 6.295208103e-06 } SecurityId Other "crsp_85216" Item crsp_85216 Weight 6.295208103460335E-06 19 { SecurityId = Other "crsp_90061"\n Weight = 8.849439676e-06 } SecurityId Other "crsp_90061" Item crsp_90061 Weight 8.849439675829064E-06 ... (more) 1 { Name = "book-to-market"\n Index = 2\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_90454"\n Weight = 8.539153991e-05 }; { SecurityId = Other "crsp_23712"\n Weight = 0.004504794958 };\n { SecurityId = Other "crsp_18... Name book-to-market Index 2 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_90454"\n Weight = 8.539153991e-05 } SecurityId Other "crsp_90454" Item crsp_90454 Weight 8.539153990698411E-05 1 { SecurityId = Other "crsp_23712"\n Weight = 0.004504794958 } SecurityId Other "crsp_23712" Item crsp_23712 Weight 0.004504794958251874 2 { SecurityId = Other "crsp_18411"\n Weight = 0.008745296825 } SecurityId Other "crsp_18411" Item crsp_18411 Weight 0.008745296825062424 3 { SecurityId = Other "crsp_42439"\n Weight = 0.000213

In [18]:
vw.Returns[..3]


index value 0 { Name = "book-to-market"\n Index = 1\n Month = 01/01/2009 00:00:00\n Return = -0.06155853096 } Name book-to-market Index 1 Month 2009-01-01 00:00:00Z Return -0.06155853095989918 1 { Name = "book-to-market"\n Index = 2\n Month = 01/01/2009 00:00:00\n Return = -0.09467724311 } Name book-to-market Index 2 Month 2009-01-01 00:00:00Z Return -0.09467724311109113 2 { Name = "book-to-market"\n Index = 3\n Month = 01/01/2009 00:00:00\n Return = -0.1636988519 } Name book-to-market Index 3 Month 2009-01-01 00:00:00Z Return -0.16369885187798322 3 { Name = "book-to-market"\n Index = 1\n Month = 01/01/2011 00:00:00\n Return = 0.01197364509 } Name book-to-market Index 1 Month 2011-01-01 00:00:00Z Return 0.011973645093246018

### Plot of value-weight returns



In [19]:
let cumulativeReturn xs =
    let sorted = xs |> List.sortBy (fun (dt, r) -> dt)
    let mutable cr = 1.0
    [ for (dt, r) in sorted do 
        cr <- cr * (1.0 + r)
        (dt, cr - 1.0) ]
let makeCumulativeChart (returns:List<PortfolioReturn>) =
    let firstObs = returns[0]
    returns
    |> List.map (fun x -> x.Month, x.Return)
    |> cumulativeReturn
    |> Chart.Line
    |> Chart.withTraceInfo(Name = $"{firstObs.Name}: {firstObs.Index}")

vw.Returns
|> List.filter (fun x -> x.Index = 1)
|> makeCumulativeChart


<!-- Plotly chart will be drawn inside this DIV -->

All the ports.



In [20]:
vw.Returns
|> List.groupBy (fun x -> x.Index)
|> List.map (fun (idx, xs) -> makeCumulativeChart xs)
|> Chart.combine


<!-- Plotly chart will be drawn inside this DIV -->

## Start of assignment

> **Task:** How many stocks are in the tercile 3 portfolio held during September 2017? Assign the result to a value named `nStocksSept2017`. Remember that this portfolio
was formed at the end of August 2017.
> 



In [21]:
let nStocksSept2017 =
    vw.Portfolios
    |> List.filter (fun p -> p.FormationMonth.Year = 2017 && p.FormationMonth.Month = 8 && p.Index = 3)
    |> List.head
    |> fun p -> p.Positions.Length

printfn "Number of stocks in the tercile 3 portfolio held during September 2017: %d" nStocksSept2017


Number of stocks in the tercile 3 portfolio held during September 2017: 1282


Tests.



In [22]:
nStocksSept2017 |> should equal 1282


> **Task:** What is the minimum and maximum weight of a stock in the tercile 3 portfolio held during September 2017? Do it for both the value and equal weight portfolios.
Assign the results to values named `vwMinSept2017`, `vwMaxSept2017`, `ewMinSept2017`, `ewMaxSept2017`.
> 



In [23]:
let vwPortfolioSept2017 =
    vw.Portfolios
    |> List.filter (fun p -> p.FormationMonth.Year = 2017 && p.FormationMonth.Month = 8 && p.Index = 3)
    |> List.head
    |> fun p -> p.Positions

let ewPortfolioSept2017 =
    ew.Portfolios
    |> List.filter (fun p -> p.FormationMonth.Year = 2017 && p.FormationMonth.Month = 8 && p.Index = 3)
    |> List.head
    |> fun p -> p.Positions

let vwMinSept2017 = vwPortfolioSept2017 |> List.minBy (fun p -> p.Weight) |> fun p -> p.Weight
let vwMaxSept2017 = vwPortfolioSept2017 |> List.maxBy (fun p -> p.Weight) |> fun p -> p.Weight
let ewMinSept2017 = ewPortfolioSept2017 |> List.minBy (fun p -> p.Weight) |> fun p -> p.Weight
let ewMaxSept2017 = ewPortfolioSept2017 |> List.maxBy (fun p -> p.Weight) |> fun p -> p.Weight

printfn "Value Weighted Portfolio:"
printfn "Minimum weight: %f" vwMinSept2017
printfn "Maximum weight: %f" vwMaxSept2017
printfn "Equal Weighted Portfolio:"
printfn "Minimum weight: %f" ewMinSept2017
printfn "Maximum weight: %f" ewMaxSept2017


Value Weighted Portfolio:
Minimum weight: 0.000000
Maximum weight: 0.064677
Equal Weighted Portfolio:
Minimum weight: 0.000780
Maximum weight: 0.000780


Tests



In [24]:
let tol = 1e-6
vwMinSept2017 |> should (equalWithin tol)  1.134675008e-07
vwMaxSept2017 |> should (equalWithin tol)  0.06467652288
ewMinSept2017 |> should (equalWithin tol)  0.0007800312012
ewMaxSept2017 |> should (equalWithin tol)  0.0007800312012


> **Task:** Plot a histogram of the Sept 2017 (formed August 2017) position weights for the stocks in the value weight tercile 3.
> 

> **Task:** Calculate the total weight put in quintile 3's top 10 positions in Sept 2017 (formed August 2017) when using value weights. Assign it to a value named `topWeightsSept2017`.
> 



In [25]:
let sept2017Tercile3VWPositions = 
    vw.Portfolios
    |> List.tryFind (fun p -> p.FormationMonth = DateTime(2017, 8, 1) && p.Index = 3)
    |> Option.map (fun p -> p.Positions)
    |> Option.defaultValue []

let weights = List.map (fun pos -> pos.Weight) sept2017Tercile3VWPositions

let histogram = Chart.Histogram(weights)
histogram


<!-- Plotly chart will be drawn inside this DIV -->

In [26]:
let top10Sept2017VWPositions =
    vw.Portfolios
    |> List.tryFind (fun p -> p.FormationMonth = DateTime(2017, 8, 1) && p.Index = 3)
    |> Option.map (fun p -> p.Positions |> List.sortByDescending (fun pos -> pos.Weight) |> List.take 10)
    |> Option.defaultValue []

let topWeightsSept2017 = List.sumBy (fun pos -> pos.Weight) top10Sept2017VWPositions
topWeightsSept2017


0.361946022511284

Tests



In [27]:
topWeightsSept2017 |> should (equalWithin tol) 0.3619460225


> **Task:** Write a function that takes a `Portfolio` as it's input and outputs a tuple of the formaiton date and the sum of the top 10 position weights. I have type hints to constrain the function type.
> 



In [28]:
//solution here
let calcTop10Weights (portfolio: Portfolio) : (DateTime * float) =
    let sorted_weights = portfolio.Positions |> List.map (fun p -> p.Weight) |> List.sort
    let top_10_sum = List.take 10 sorted_weights |> List.sum
    (portfolio.FormationMonth, top_10_sum)


tests



In [29]:
// Portfolio with 10 test positions
let testPortfolio =
    { FormationMonth = DateTime(1999,1,1)
      Name = "test"
      Index = 1
      Positions = [ for i in 1..20 do { SecurityId = Other "test"; Weight = 1./20.} ] }

let testPortfolioDate, testPortfolioWeight = testPortfolio |> calcTop10Weights

testPortfolioDate |> should equal (DateTime(1999,1,1))
testPortfolioWeight |> should (equalWithin tol) 0.5


> **Task:** Using the value-weight strategy, calculate the total weight put in quintile 3's top 10 positions every month. Assign it to a value named `topWeights` that has type `list<DateTime * float>` where the first thing in the tuple is the formation month and the second thing is the sum of the top 10 position weights.
> 



In [37]:
// Solution here
let topWeights= 
    vw.Portfolios
    |> List.filter (fun p -> p.Index = 3)
    |> List.map (fun p -> 
        let topTenPositions = p.Positions |> List.sortByDescending (fun pos -> pos.Weight) |> List.take 10
        let sumOfWeights = topTenPositions |> List.sumBy (fun pos -> pos.Weight)
        (p.FormationMonth, sumOfWeights))

tests



In [38]:
topWeights |> shouldHaveLength 252
topWeights |> should be ofExactType<list<DateTime * float>>
topWeights
|> List.averageBy (fun (dt, w) -> w)
|> should (equalWithin tol) 0.3174428516


> **Task:** Plot a line chart of `topWeights` that shows how the top 10 weights evolves over the sample period.
> 



In [49]:
let x =
    topWeights
    |>List.sortBy(fun x->x.Item1.Year,x.Item1.Month)
    |> Chart.Line
    |> Chart.withTitle "Top 10 weights"
    |> Chart.withXAxisStyle("Month")
    |> Chart.withYAxisStyle("Weight")
x

<!-- Plotly chart will be drawn inside this DIV -->